In [20]:
import pandas as pd
import numpy as np
import random

In [21]:
print('A' > '0')
print('a' > '0')
print('a' > 'A')

True
True
True


In [22]:
waypoint_distances = {}
waypoint_durations = {}
all_waypoints = set()

waypoint_data = pd.read_csv("my-waypoints-dist-dur.tsv", sep="\t")

for i, row in waypoint_data.iterrows():
    waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
    waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
    all_waypoints.update([row.waypoint1, row.waypoint2])
    
print(waypoint_data)

                                   waypoint1  \
0                         Huk Aveny 35, Oslo   
1                        Gronnegata 11, Oslo   
2                       Nobels gate 32, Oslo   
3                         Bygdoy, Oslo, Oslo   
4                         Kongeveien 5, Oslo   
5                                  nan, Oslo   
6              Kirsten Flagstads Pl. 1, Oslo   
7              Kirsten Flagstads Pl. 1, Oslo   
8                   Bygdoeynesveien 39, Oslo   
9   Universitetsgaten 13 (city centre), Oslo   
10                        Bygdoy, Oslo, Oslo   
11                        Huk Aveny 35, Oslo   
12             Kirsten Flagstads Pl. 1, Oslo   
13                        Huk Aveny 35, Oslo   
14                      Nobels gate 32, Oslo   
15  Universitetsgaten 13 (city centre), Oslo   
16  Universitetsgaten 13 (city centre), Oslo   
17                        Huk Aveny 35, Oslo   
18             Kirsten Flagstads Pl. 1, Oslo   
19                        Huk Aveny 35, 

In [13]:
homebase = 'Gronnegata 11, Oslo'


In [15]:
my_tuple = ('Nobels gate 32, Oslo', 'Bygdoy, Oslo, Oslo', 'Huk Aveny 35, Oslo', 
            'Kongeveien 5, Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Museumsveien 10, Oslo',
            'Universitetsgaten 13 (city centre), Oslo', 'Kirsten Flagstads Pl. 1, Oslo')
new_tupla = (homebase,) + my_tuple
print(new_tupla)

('Gronnegata 11, Oslo', 'Nobels gate 32, Oslo', 'Bygdoy, Oslo, Oslo', 'Huk Aveny 35, Oslo', 'Kongeveien 5, Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Museumsveien 10, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'Kirsten Flagstads Pl. 1, Oslo')


In [39]:
def compute_fitness(homebase, solution):
    """
        This function returns the total distance traveled on the current road trip.

        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """

    solution_fitness = 0.0
    solution = (homebase,) + solution

    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]

    return solution_fitness


def generate_random_agent(homebase):
    """
        Creates a random road trip from the waypoints.
    """

    new_random_agent = list(all_waypoints)
    new_random_agent.remove(homebase)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)


def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.

        A point mutation swaps the order of two waypoints in the road trip.
    """

    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)

    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]

    return tuple(agent_genome)


def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.

        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """

    agent_genome = list(agent_genome)

    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)

    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]

    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]

    return tuple(agent_genome)


def generate_random_population(homebase, pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """

    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent(homebase))
    return random_population


def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.

        `generations` and `population_size` must be a multiple of 10.
    """

    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)

    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(homebase, population_size)
    # print(population)

    # For `generations` number of repetitions...
    for generation in range(generations):

        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(homebase, agent_genome)

        # Take the top 10% shortest road trips and produce offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):

            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                complete_agent_genome = (homebase,) + agent_genome
                print(complete_agent_genome)
                print("")

            # Create 1 exact copy of each of the top road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-3 point mutations
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, 3))

            # Create 7 offspring with a single shuffle mutation
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population

In [40]:
run_genetic_algorithm(generations=5000, population_size=100)

Generation 0 best: 29026 | Unique genomes: 100
('Gronnegata 11, Oslo', 'Bygdoy, Oslo, Oslo', 'Huk Aveny 35, Oslo', 'Museumsveien 10, Oslo', 'Bygdoeynesveien 39, Oslo', 'Kongeveien 5, Oslo', 'Nobels gate 32, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo')

Generation 500 best: 27277 | Unique genomes: 76
('Gronnegata 11, Oslo', 'nan, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Nobels gate 32, Oslo', 'Museumsveien 10, Oslo', 'Bygdoeynesveien 39, Oslo', 'Huk Aveny 35, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo')

Generation 1000 best: 27277 | Unique genomes: 69
('Gronnegata 11, Oslo', 'nan, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Nobels gate 32, Oslo', 'Museumsveien 10, Oslo', 'Bygdoeynesveien 39, Oslo', 'Huk Aveny 35, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo')

Generation 1500 best: 27277 | Unique genomes: 68
('Gronnegata 11, Oslo', 'nan, Osl

In [33]:
generations = 5000
population_size = 100
homebase = 'Gronnegata 11, Oslo'

population_subset_size = int(population_size / 10.)
generations_10pct = int(generations / 10.)
# print(population_subset_size) -> 10
# print(generations_10pct) -> 500

# Create a random population of `population_size` number of solutions.
# population = generate_random_population(population_size)

random_population = []
for agent in range(population_size):
    
    # Creates a random road trip from the waypoints.
    # random_population.append(generate_random_agent())
    
    new_random_agent = list(all_waypoints)
    new_random_agent.remove(homebase)
    random.shuffle(new_random_agent)
    # print(new_random_agent)
    
    # return tuple(new_random_agent)
    random_population.append(tuple(new_random_agent))
    
# return random_population
population = random_population
# print(population)

In [34]:
# for generation in range(generations):
# Taking the first iteration
generation = 0

# Compute the fitness of the entire current population
population_fitness = {}

for agent_genome in population:
    if agent_genome in population_fitness:
        continue

    # population_fitness[agent_genome] = compute_fitness(agent_genome)
    
    # def compute_fitness(solution):
    # This function returns the total distance traveled on the current road trip.

    # The genetic algorithm will favor road trips that have shorter
    # total distances traveled.
    solution = agent_genome

    solution_fitness = 0.0
    solution = (homebase,) + solution

    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        # print(solution_fitness)
        
    # print(solution_fitness)
    # print()
    
    population_fitness[agent_genome] = solution_fitness
    print(population_fitness[agent_genome])

53958.0
41555.0
45994.0
39941.0
50668.0
50322.0
45800.0
48174.0
36598.0
37783.0
49620.0
39542.0
47309.0
37536.0
46170.0
53410.0
37988.0
38793.0
52291.0
39484.0
42349.0
53173.0
41259.0
50470.0
45469.0
43571.0
48607.0
38479.0
54113.0
47521.0
42940.0
43011.0
39572.0
38103.0
41046.0
33474.0
36420.0
38166.0
48245.0
44349.0
38410.0
52345.0
43837.0
43479.0
50857.0
57369.0
46707.0
50234.0
37170.0
50622.0
36436.0
53932.0
48955.0
46565.0
49357.0
42353.0
52940.0
38303.0
44362.0
47109.0
45063.0
56806.0
38200.0
36765.0
35187.0
40196.0
44404.0
41695.0
39434.0
38389.0
50430.0
38638.0
34070.0
42882.0
34729.0
56855.0
45755.0
57537.0
40283.0
57639.0
41255.0
52646.0
49788.0
50472.0
56701.0
34179.0
32292.0
32492.0
42033.0
49784.0
45198.0
57260.0
44768.0
56964.0
45588.0
48031.0
56559.0
41609.0
48032.0
28884.0


In [36]:
# The best trips: we get new paths starting from these
best_trips = sorted(population_fitness, key=population_fitness.get)[:population_subset_size]
for i in best_trips:
    print(i)
    print(population_fitness[i])

('Bygdoeynesveien 39, Oslo', 'Museumsveien 10, Oslo', 'Huk Aveny 35, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo', 'Nobels gate 32, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'nan, Oslo', 'Universitetsgaten 13 (city centre), Oslo')
28884.0
('Kirsten Flagstads Pl. 1, Oslo', 'nan, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'Museumsveien 10, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo', 'Bygdoeynesveien 39, Oslo', 'Huk Aveny 35, Oslo', 'Nobels gate 32, Oslo')
32292.0
('Kirsten Flagstads Pl. 1, Oslo', 'Museumsveien 10, Oslo', 'Huk Aveny 35, Oslo', 'Bygdoy, Oslo, Oslo', 'Bygdoeynesveien 39, Oslo', 'Kongeveien 5, Oslo', 'Nobels gate 32, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo')
32492.0
('Universitetsgaten 13 (city centre), Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Huk Aveny 35, Oslo', 'Nobels gate 32, Oslo', 'Kongeveien 5, Oslo', 'Bygdoy, Oslo, Oslo', 'Museumsveien 10, Oslo')
33474.0
('nan, Oslo', 'Huk Aveny 35, Osl

In [35]:
# Take the top 10% shortest road trips and produce offspring each from them
new_population = []
for rank, agent_genome in enumerate(sorted(population_fitness,
                                           key=population_fitness.get)[:population_subset_size]):

    if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
        print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                               population_fitness[agent_genome],
                                                               len(population_fitness)))
        print(agent_genome)
        print("")

    # Create 1 exact copy of each of the top road trips
    new_population.append(agent_genome)

    # Create 2 offspring with 1-3 point mutations
    print("Mutate agent:")
    for offspring in range(2):
        # new_population.append(mutate_agent(agent_genome, 3))
        # def mutate_agent(agent_genome, max_mutations=3):
        # Applies 1 - `max_mutations` point mutations to the given road trip.
        # A point mutation swaps the order of two waypoints in the road trip.
        max_mutations = 3

        agent_genome = list(agent_genome)
        print(agent_genome)
        num_mutations = random.randint(1, max_mutations)

        for mutation in range(num_mutations):
            swap_index1 = random.randint(0, len(agent_genome) - 1)
            swap_index2 = swap_index1

            while swap_index1 == swap_index2:
                swap_index2 = random.randint(0, len(agent_genome) - 1)

            agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
        
        new_population.append(tuple(agent_genome))
        print(tuple(agent_genome))
        print()


    # Create 7 offspring with a single shuffle mutation
    print("Shuffle_mutation:")
    for offspring in range(7):
        # new_population.append(shuffle_mutation(agent_genome))
        # def shuffle_mutation(agent_genome):
        # Applies a single shuffle mutation to the given road trip.
        # A shuffle mutation takes a random sub-section of the road trip
        # and moves it to another location in the road trip.
    
        agent_genome = list(agent_genome)
        print(agent_genome)

        start_index = random.randint(0, len(agent_genome) - 1)
        length = random.randint(2, 20)

        genome_subset = agent_genome[start_index:start_index + length]
        agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]

        insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
        agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]

        new_population.append(tuple(agent_genome))
        print(tuple(agent_genome))
        print()

# Replace the old population with the new population of offspring
for i in range(len(population))[::-1]:
    del population[i]

population = new_population

Generation 0 best: 27111 | Unique genomes: 100
('Huk Aveny 35, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Nobels gate 32, Oslo', 'Kongeveien 5, Oslo', 'Bygdoy, Oslo, Oslo', 'Museumsveien 10, Oslo', 'Bygdoeynesveien 39, Oslo')

Mutate agent:
['Huk Aveny 35, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Nobels gate 32, Oslo', 'Kongeveien 5, Oslo', 'Bygdoy, Oslo, Oslo', 'Museumsveien 10, Oslo', 'Bygdoeynesveien 39, Oslo']
('Museumsveien 10, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo', 'Huk Aveny 35, Oslo', 'Nobels gate 32, Oslo')

['Museumsveien 10, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo', 'Huk Aveny 35, Oslo', 'Nobels gate 32, Oslo']
(